In [2]:
import pandas as pd
import numpy as np
import folium
import branca.colormap as cm

In [2]:
def plot_map(df, column_cluster="cluster", icons_same_color=False, zoom_start=7):
    colors = ['blue','red','green','orange','pink','black','lightblue','purple','gray','cadetblue']
    colors_icons = ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
                    'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
                    'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
                    'gray', 'black', 'lightgray']

    center = [np.mean([np.mean(df.lat_x), np.mean(df.lat_y),]),
              np.mean([np.mean(df.long_x), np.mean(df.long_y),]),
             ]
    map_sp = folium.Map(location=center, zoom_start=zoom_start, width='100%', height="100%")

    for index, d in df.iterrows():

        location = [d['lat_x'], d['long_x']]
        color_type = {"collect":"green", "delivery":"blue", "deliveryMan":"red"}
        popup = {"collect":f"Origin \nId: {d['Id']} \nType: {d['type']} \nType Neighbor: {d['type_neighbor']}",
                 "delivery":f"Destination \nId: {d['Id']} \nType: {d['type']} \nType Neighbor: {d['type_neighbor']}",
                 "deliveryMan":f"DeliveryMan \nId: {d['Id']} \nType: {d['type']} \nType Neighbor: {d['type_neighbor']}"}
        
        folium.Marker(location,
                      popup=popup[d["type"]],
                      icon=folium.Icon(color=color_type[d["type"]]),
                     ).add_to(map_sp)

        folium.PolyLine([[d['lat_x'], d['long_x']], [d['lat_y'], d['long_y']]],
                        weight=2.5,
                        color="black",
                       ).add_to(map_sp)

    legend_html =   '''
                    <div style="position: fixed; 
                                bottom: 50px; right: 50px; width: 150px; height: 90px; 
                                border:2px solid grey; z-index:9999; font-size:14px;
                                ">&nbsp; Legend <br>
                                  &nbsp; Origin &nbsp; <i class="fa fa-map-marker fa-2x" style="color:green"></i><br>
                                  &nbsp; Destination &nbsp; <i class="fa fa-map-marker fa-2x" style="color:blue"></i>
                    </div>
                    '''

    map_sp.get_root().html.add_child(folium.Element(legend_html))

    display(map_sp)

# Método
- construção do grafo ligando cada ponto aos k pontos mais próximos
    - inicialmente k = 5

In [11]:
k = 5

In [12]:
orders = pd.read_csv("..\\..\\Delivery-Clustering\\Notebooks\\Bases\\04_Resultado_da_Clusterização_e_Matching.csv")
orders.head()

,Id,TamanhoPacote__c,CollectPoint__Latitude__s,CollectPoint__Longitude__s,DeliveryPoint__Latitude__s,DeliveryPoint__Longitude__s,cluster,id_deliveryman,car_capacity__c,DeliveryManLocation__Latitude__s,DeliveryManLocation__Longitude__s
0,90e4d4f7e2221bf7f81a4873e3161b96,Pequeno,-23.516334,-46.546652,-23.664256,-46.688579,0,82413,Média,-23.597327,-46.595990
1,9d3a10d6a5aab8ca343a21580c404b69,Grande,-23.581770,-46.508625,-23.668184,-46.645959,3,86788,Alta,-23.630328,-46.640746
2,8ebaf041cc8d84dc5436959bab95f2ce,Pequeno,-23.477819,-46.683036,-23.690930,-46.363821,0,82413,Média,-23.597327,-46.595990
3,17e604bb550e265b46ebefe61430751b,Pequeno,-24.030773,-46.487485,-23.664503,-47.581761,6,253167,Baixa,-23.973751,-46.371917
4,44f77123c08b319dff9a40cb561a6169,Pequeno,-23.718206,-46.692554,-20.647806,-50.225672,1,96477,Baixa,-23.595924,-46.669864


In [13]:
clusters = np.unique(orders["cluster"])
clusters

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)

In [14]:
cluster_1 = orders.query(f"cluster == {clusters[1]}")
cluster_1

,Id,TamanhoPacote__c,CollectPoint__Latitude__s,CollectPoint__Longitude__s,DeliveryPoint__Latitude__s,DeliveryPoint__Longitude__s,cluster,id_deliveryman,car_capacity__c,DeliveryManLocation__Latitude__s,DeliveryManLocation__Longitude__s
4,44f77123c08b319dff9a40cb561a6169,Pequeno,-23.718206,-46.692554,-20.647806,-50.225672,1,96477,Baixa,-23.595924,-46.669864
17,56a726fa0f40d3678696e62aaed9690d,Pequeno,-23.480948,-46.589204,-20.269329,-50.548525,1,96477,Baixa,-23.595924,-46.669864
41,b474873efc88629c172f639f9b47138d,Pequeno,-23.593297,-46.552484,-21.218322,-50.400362,1,96477,Baixa,-23.595924,-46.669864


In [15]:
def _calculateCartesianDistance(points):
    
    for i, r in points.iterrows():
        xA_less_xB = points["lat"] - r["lat"]
        yA_less_yB = points["long"] - r["long"]
        points[r["Id"]+"_"+r["type"]] = np.sqrt((xA_less_xB**2) + (yA_less_yB**2))
    
    return points

In [25]:
collectPoint = (cluster_1[["Id", "CollectPoint__Latitude__s", "CollectPoint__Longitude__s"]]
                .rename(columns={"CollectPoint__Latitude__s":"lat",
                                 "CollectPoint__Longitude__s":"long"})
               )
collectPoint["type"] = "collect"

deliveryPoint = (cluster_1[["Id", "DeliveryPoint__Latitude__s", "DeliveryPoint__Longitude__s"]]
                 .rename(columns={"DeliveryPoint__Latitude__s":"lat",
                                  "DeliveryPoint__Longitude__s":"long"})
                )
deliveryPoint["type"] = "delivery"

deliveryManLocation = (cluster_1[["Id", "DeliveryManLocation__Latitude__s", "DeliveryManLocation__Longitude__s"]]
                       .rename(columns={"DeliveryManLocation__Latitude__s":"lat",
                                        "DeliveryManLocation__Longitude__s":"long"})
                      )
deliveryManLocation["type"] = "deliveryMan"

points = pd.concat([collectPoint, deliveryPoint,
                    deliveryManLocation
                   ])

points_2 = points.copy()

for i, r in points_2.iterrows():
    xA_less_xB = points_2["lat"] - r["lat"]
    yA_less_yB = points_2["long"] - r["long"]
    points_2[r["Id"]+"_"+r["type"]] = np.sqrt((xA_less_xB**2) + (yA_less_yB**2))
    
points_2 = (
    points_2
    .melt(id_vars=["Id", "lat", "long", "type"])
    .query("value > 0")
)

points_2["rw"] = (
    points_2
    .sort_values("value", ascending=True)
    .groupby(["Id", "type"])
    .cumcount() + 1
)

points_2 = points_2.query("rw in [1, 2, 3, 4, 5]")

points_2["Id_neighbor"] = points_2.variable.str.extract(pat=r"(.*)(_(collect|delivery$|deliveryMan))")[0]
points_2["type_neighbor"] = points_2.variable.str.extract(pat=r"(.*_)((collect|delivery$|deliveryMan))")[1]

points_2 = pd.merge(
    points_2,
    points.rename(columns={"Id":"Id_neighbor", "type":"type_neighbor"}).set_index("Id_neighbor"),
    left_on=["Id_neighbor", "type_neighbor"],
    right_on=["Id_neighbor", "type_neighbor"],
    how="inner",
)

deliveryMan = points_2.query("type == 'deliveryMan'").sort_values(by=["value"], ascending=True).iloc[:1]

points_3 = pd.concat([deliveryMan,
                      points_2.query("type != 'deliveryMan'")
                     ])

points_3

,Id,lat_x,long_x,type,variable,value,rw,Id_neighbor,type_neighbor,lat_y,long_y
14,44f77123c08b319dff9a40cb561a6169,-23.595924,-46.669864,deliveryMan,b474873efc88629c172f639f9b47138d_collect,0.117410,1,b474873efc88629c172f639f9b47138d,collect,-23.593297,-46.552484
0,56a726fa0f40d3678696e62aaed9690d,-23.480948,-46.589204,collect,44f77123c08b319dff9a40cb561a6169_collect,0.258791,5,44f77123c08b319dff9a40cb561a6169,collect,-23.718206,-46.692554
1,b474873efc88629c172f639f9b47138d,-23.593297,-46.552484,collect,44f77123c08b319dff9a40cb561a6169_collect,0.187675,5,44f77123c08b319dff9a40cb561a6169,collect,-23.718206,-46.692554
5,44f77123c08b319dff9a40cb561a6169,-23.718206,-46.692554,collect,56a726fa0f40d3678696e62aaed9690d_collect,0.258791,5,56a726fa0f40d3678696e62aaed9690d,collect,-23.480948,-46.589204
6,b474873efc88629c172f639f9b47138d,-23.593297,-46.552484,collect,56a726fa0f40d3678696e62aaed9690d_collect,0.118197,4,56a726fa0f40d3678696e62aaed9690d,collect,-23.480948,-46.589204
7,44f77123c08b319dff9a40cb561a6169,-20.647806,-50.225672,delivery,56a726fa0f40d3678696e62aaed9690d_collect,4.609837,3,56a726fa0f40d3678696e62aaed9690d,collect,-23.480948,-46.589204
8,56a726fa0f40d3678696e62aaed9690d,-20.269329,-50.548525,delivery,56a726fa0f40d3678696e62aaed9690d_collect,5.098110,3,56a726fa0f40d3678696e62aaed9690d,collect,-23.480948,-46.589204
12,44f77123c08b319dff9a40cb561a6169,-23.718206,-46.692554,collect,b474873efc88629c172f639f9b47138d_collect,0.187675,4,b474873efc88629c172f639f9b47138d,collect,-23.593297,-46.552484
13,56a726fa0f40d3678696e62aaed9690d,-23.480948,-46.589204,collect,b474873efc88629c172f639f9b47138d_collect,0.118197,1,b474873efc88629c172f639f9b47138d,collect,-23.593297,-46.552484
17,56a726fa0f40d3678696e62aaed9690d,-20.269329,-50.548525,delivery,44f77123c08b319dff9a40cb561a6169_delivery,0.497472,1,44f77123c08b319dff9a40cb561a6169,delivery,-20.647806,-50.225672


In [41]:
points_3 = points_3.melt(id_vars=["origin_id", "type", "cluster"], var_name="variable")
points_3["Id_neighbor"] = points_3.variable.str.extract(pat=r"(.*)(_(collect|delivery$|deliveryMan))")[0]
points_3["type_neighbor"] = points_3.variable.str.extract(pat=r"(.*_)((collect|delivery$|deliveryMan))")[1]

In [42]:
points_3

,origin_id,type,cluster,variable,value,Id_neighbor,type_neighbor
0,44f77123c08b319dff9a40cb561a6169_collect,collect,1,44f77123c08b319dff9a40cb561a6169_collect,NaN,44f77123c08b319dff9a40cb561a6169,collect
1,44f77123c08b319dff9a40cb561a6169_delivery,delivery,1,44f77123c08b319dff9a40cb561a6169_collect,NaN,44f77123c08b319dff9a40cb561a6169,collect
2,44f77123c08b319dff9a40cb561a6169_deliveryMan,deliveryMan,1,44f77123c08b319dff9a40cb561a6169_collect,NaN,44f77123c08b319dff9a40cb561a6169,collect
3,56a726fa0f40d3678696e62aaed9690d_collect,collect,1,44f77123c08b319dff9a40cb561a6169_collect,0.258791,44f77123c08b319dff9a40cb561a6169,collect
4,56a726fa0f40d3678696e62aaed9690d_delivery,delivery,1,44f77123c08b319dff9a40cb561a6169_collect,NaN,44f77123c08b319dff9a40cb561a6169,collect
...,...,...,...,...,...,...,...
58,44f77123c08b319dff9a40cb561a6169_deliveryMan,deliveryMan,1,b474873efc88629c172f639f9b47138d_deliveryMan,NaN,b474873efc88629c172f639f9b47138d,deliveryMan
59,56a726fa0f40d3678696e62aaed9690d_collect,collect,1,b474873efc88629c172f639f9b47138d_deliveryMan,0.140448,b474873efc88629c172f639f9b47138d,deliveryMan
60,56a726fa0f40d3678696e62aaed9690d_delivery,delivery,1,b474873efc88629c172f639f9b47138d_deliveryMan,NaN,b474873efc88629c172f639f9b47138d,deliveryMan
61,b474873efc88629c172f639f9b47138d_collect,collect,1,b474873efc88629c172f639f9b47138d_deliveryMan,0.117410,b474873efc88629c172f639f9b47138d,deliveryMan


In [35]:
collectPoint = (
    cluster_1[["Id", "cluster", "CollectPoint__Latitude__s", "CollectPoint__Longitude__s"]]
    .rename(columns={"CollectPoint__Latitude__s":"lat",
                     "CollectPoint__Longitude__s":"long"})
)
collectPoint["type"] = "collect"

deliveryPoint = (cluster_1[["Id", "cluster", "DeliveryPoint__Latitude__s", "DeliveryPoint__Longitude__s"]]
                 .rename(columns={"DeliveryPoint__Latitude__s":"lat",
                                  "DeliveryPoint__Longitude__s":"long"})
                )
deliveryPoint["type"] = "delivery"

deliveryManLocation = (cluster_1[["Id", "cluster", "DeliveryManLocation__Latitude__s", "DeliveryManLocation__Longitude__s"]]
                       .rename(columns={"DeliveryManLocation__Latitude__s":"lat",
                                        "DeliveryManLocation__Longitude__s":"long"})
                      )
deliveryManLocation["type"] = "deliveryMan"

points = pd.concat([collectPoint,
                    deliveryPoint,
                    deliveryManLocation
                   ])

points_2 = points.copy()

points_2 = _calculateCartesianDistance(points_2)
    
points_2 = (
    points_2
    .melt(id_vars=["Id", "type", "cluster"])
    .query("value > 0")
)

points_2["rw"] = (
    points_2
    .sort_values("value", ascending=True)
    .groupby(["Id", "type"])
    .cumcount() + 1
)

# points_2 = points_2.query("rw in [1, 2, 3, 4, 5, ..., k]")
points_2 = points_2.query(f"rw in {[i for i in range(k+1)]}")

points_2["origin_id"] = points_2["Id"] + "_" + points_2["type"]
points_2 = points_2.rename(columns={"variable":"destination_id","value":"distance"})
points_2 = points_2[["origin_id", "type", "destination_id", "distance", "cluster"]]

# DeliveryMan terá apenas uma única direção possível no grafo,
# sendo ela a origem da ordem mais próxima a ele
deliveryMan = (
    points_2
    .query("type == 'deliveryMan'")
    .sort_values(by=["distance"], ascending=True)
    .iloc[:1]
)

points_3 = pd.concat([deliveryMan,
                      points_2.query("type != 'deliveryMan'")
                    ])

# Será retornado uma matriz de distâncias
points_3 = points_3.pivot(index=["origin_id",
                                 "type",
                                 "cluster"],
                          columns="destination_id",
                          values="distance"
                        )

points_3 = points_3.reset_index()

points_3

destination_id,origin_id,type,cluster,44f77123c08b319dff9a40cb561a6169_collect,44f77123c08b319dff9a40cb561a6169_delivery,44f77123c08b319dff9a40cb561a6169_deliveryMan,56a726fa0f40d3678696e62aaed9690d_collect,56a726fa0f40d3678696e62aaed9690d_delivery,56a726fa0f40d3678696e62aaed9690d_deliveryMan,b474873efc88629c172f639f9b47138d_collect,b474873efc88629c172f639f9b47138d_delivery,b474873efc88629c172f639f9b47138d_deliveryMan
0,44f77123c08b319dff9a40cb561a6169_collect,collect,1,NaN,NaN,0.124369,0.258791,NaN,0.124369,0.187675,NaN,0.124369
1,44f77123c08b319dff9a40cb561a6169_delivery,delivery,1,NaN,NaN,4.619001,4.609837,0.497472,4.619001,NaN,0.596662,NaN
2,44f77123c08b319dff9a40cb561a6169_deliveryMan,deliveryMan,1,NaN,NaN,NaN,NaN,NaN,NaN,0.117410,NaN,NaN
3,56a726fa0f40d3678696e62aaed9690d_collect,collect,1,0.258791,NaN,0.140448,NaN,NaN,0.140448,0.118197,NaN,0.140448
4,56a726fa0f40d3678696e62aaed9690d_delivery,delivery,1,NaN,0.497472,5.109819,5.098110,NaN,5.109819,NaN,0.960489,NaN
5,b474873efc88629c172f639f9b47138d_collect,collect,1,0.187675,NaN,0.117410,0.118197,NaN,0.117410,NaN,NaN,0.117410
6,b474873efc88629c172f639f9b47138d_delivery,delivery,1,NaN,0.596662,4.423755,NaN,0.960489,4.423755,NaN,NaN,4.423755


In [8]:
plot_map(points_3, zoom_start=9)

# Passo a passo com um único `Id`

In [17]:
test = points.query(f"Id == '64b7288026efc4a12f4e3351f03f2ac4'")
test

,Id,lat,long,type
37,64b7288026efc4a12f4e3351f03f2ac4,-21.105158,-47.785773,collect
37,64b7288026efc4a12f4e3351f03f2ac4,-22.147359,-48.526709,delivery
37,64b7288026efc4a12f4e3351f03f2ac4,-23.115526,-46.552918,deliveryMan


In [18]:
test_2 = test.copy()

In [19]:
for i, r in test_2.iterrows():
    xA_less_xB = test_2["lat"] - r["lat"]
    yA_less_yB = test_2["long"] - r["long"]
    test_2[r["Id"] + "_" + r["type"]] = np.sqrt((xA_less_xB**2) + (yA_less_yB**2))
    
test_2

,Id,lat,long,type,64b7288026efc4a12f4e3351f03f2ac4_collect,64b7288026efc4a12f4e3351f03f2ac4_delivery,64b7288026efc4a12f4e3351f03f2ac4_deliveryMan
37,64b7288026efc4a12f4e3351f03f2ac4,-21.105158,-47.785773,collect,0.000000,1.278737,2.358286
37,64b7288026efc4a12f4e3351f03f2ac4,-22.147359,-48.526709,delivery,1.278737,0.000000,2.198454
37,64b7288026efc4a12f4e3351f03f2ac4,-23.115526,-46.552918,deliveryMan,2.358286,2.198454,0.000000


In [20]:
test_2 = (
    test_2
    .melt(id_vars=["Id", "lat", "long", "type"])
    .query("value > 0")
    .sort_values("value", ascending=True)
)
test_2 

,Id,lat,long,type,variable,value
1,64b7288026efc4a12f4e3351f03f2ac4,-22.147359,-48.526709,delivery,64b7288026efc4a12f4e3351f03f2ac4_collect,1.278737
3,64b7288026efc4a12f4e3351f03f2ac4,-21.105158,-47.785773,collect,64b7288026efc4a12f4e3351f03f2ac4_delivery,1.278737
5,64b7288026efc4a12f4e3351f03f2ac4,-23.115526,-46.552918,deliveryMan,64b7288026efc4a12f4e3351f03f2ac4_delivery,2.198454
7,64b7288026efc4a12f4e3351f03f2ac4,-22.147359,-48.526709,delivery,64b7288026efc4a12f4e3351f03f2ac4_deliveryMan,2.198454
2,64b7288026efc4a12f4e3351f03f2ac4,-23.115526,-46.552918,deliveryMan,64b7288026efc4a12f4e3351f03f2ac4_collect,2.358286
6,64b7288026efc4a12f4e3351f03f2ac4,-21.105158,-47.785773,collect,64b7288026efc4a12f4e3351f03f2ac4_deliveryMan,2.358286


In [21]:
test_2["rw"] = (
    test_2
    .sort_values("value", ascending=True)
    .groupby(["Id", "type"])
    .cumcount() + 1
)
test_2

,Id,lat,long,type,variable,value,rw
1,64b7288026efc4a12f4e3351f03f2ac4,-22.147359,-48.526709,delivery,64b7288026efc4a12f4e3351f03f2ac4_collect,1.278737,1
3,64b7288026efc4a12f4e3351f03f2ac4,-21.105158,-47.785773,collect,64b7288026efc4a12f4e3351f03f2ac4_delivery,1.278737,1
5,64b7288026efc4a12f4e3351f03f2ac4,-23.115526,-46.552918,deliveryMan,64b7288026efc4a12f4e3351f03f2ac4_delivery,2.198454,1
7,64b7288026efc4a12f4e3351f03f2ac4,-22.147359,-48.526709,delivery,64b7288026efc4a12f4e3351f03f2ac4_deliveryMan,2.198454,2
2,64b7288026efc4a12f4e3351f03f2ac4,-23.115526,-46.552918,deliveryMan,64b7288026efc4a12f4e3351f03f2ac4_collect,2.358286,2
6,64b7288026efc4a12f4e3351f03f2ac4,-21.105158,-47.785773,collect,64b7288026efc4a12f4e3351f03f2ac4_deliveryMan,2.358286,2


In [22]:
k = 5
l = [i for i in range(k)]
test_2.query(f"rw in {l}")

,Id,lat,long,type,variable,value,rw
1,64b7288026efc4a12f4e3351f03f2ac4,-22.147359,-48.526709,delivery,64b7288026efc4a12f4e3351f03f2ac4_collect,1.278737,1
3,64b7288026efc4a12f4e3351f03f2ac4,-21.105158,-47.785773,collect,64b7288026efc4a12f4e3351f03f2ac4_delivery,1.278737,1
5,64b7288026efc4a12f4e3351f03f2ac4,-23.115526,-46.552918,deliveryMan,64b7288026efc4a12f4e3351f03f2ac4_delivery,2.198454,1
7,64b7288026efc4a12f4e3351f03f2ac4,-22.147359,-48.526709,delivery,64b7288026efc4a12f4e3351f03f2ac4_deliveryMan,2.198454,2
2,64b7288026efc4a12f4e3351f03f2ac4,-23.115526,-46.552918,deliveryMan,64b7288026efc4a12f4e3351f03f2ac4_collect,2.358286,2
6,64b7288026efc4a12f4e3351f03f2ac4,-21.105158,-47.785773,collect,64b7288026efc4a12f4e3351f03f2ac4_deliveryMan,2.358286,2


In [23]:
test_2["Id_neighbor"] = test_2.variable.str.extract(pat=r"(.*)(_(collect|delivery$|deliveryMan))")[0]
test_2["type_neighbor"] = test_2.variable.str.extract(pat=r"(.*_)((collect|delivery$|deliveryMan))")[1]
test_2

,Id,lat,long,type,variable,value,rw,Id_neighbor,type_neighbor
1,64b7288026efc4a12f4e3351f03f2ac4,-22.147359,-48.526709,delivery,64b7288026efc4a12f4e3351f03f2ac4_collect,1.278737,1,64b7288026efc4a12f4e3351f03f2ac4,collect
3,64b7288026efc4a12f4e3351f03f2ac4,-21.105158,-47.785773,collect,64b7288026efc4a12f4e3351f03f2ac4_delivery,1.278737,1,64b7288026efc4a12f4e3351f03f2ac4,delivery
5,64b7288026efc4a12f4e3351f03f2ac4,-23.115526,-46.552918,deliveryMan,64b7288026efc4a12f4e3351f03f2ac4_delivery,2.198454,1,64b7288026efc4a12f4e3351f03f2ac4,delivery
7,64b7288026efc4a12f4e3351f03f2ac4,-22.147359,-48.526709,delivery,64b7288026efc4a12f4e3351f03f2ac4_deliveryMan,2.198454,2,64b7288026efc4a12f4e3351f03f2ac4,deliveryMan
2,64b7288026efc4a12f4e3351f03f2ac4,-23.115526,-46.552918,deliveryMan,64b7288026efc4a12f4e3351f03f2ac4_collect,2.358286,2,64b7288026efc4a12f4e3351f03f2ac4,collect
6,64b7288026efc4a12f4e3351f03f2ac4,-21.105158,-47.785773,collect,64b7288026efc4a12f4e3351f03f2ac4_deliveryMan,2.358286,2,64b7288026efc4a12f4e3351f03f2ac4,deliveryMan


In [24]:
test_2 = pd.merge(
    test_2,
    test.rename(columns={"Id":"Id_neighbor", "type":"type_neighbor"}).set_index("Id_neighbor"),
    left_on=["Id_neighbor", "type_neighbor"],
    right_on=["Id_neighbor", "type_neighbor"],
    how="inner",
)
test_2.sort_values(["Id", "type", "rw"])

,Id,lat_x,long_x,type,variable,value,rw,Id_neighbor,type_neighbor,lat_y,long_y
2,64b7288026efc4a12f4e3351f03f2ac4,-21.105158,-47.785773,collect,64b7288026efc4a12f4e3351f03f2ac4_delivery,1.278737,1,64b7288026efc4a12f4e3351f03f2ac4,delivery,-22.147359,-48.526709
5,64b7288026efc4a12f4e3351f03f2ac4,-21.105158,-47.785773,collect,64b7288026efc4a12f4e3351f03f2ac4_deliveryMan,2.358286,2,64b7288026efc4a12f4e3351f03f2ac4,deliveryMan,-23.115526,-46.552918
0,64b7288026efc4a12f4e3351f03f2ac4,-22.147359,-48.526709,delivery,64b7288026efc4a12f4e3351f03f2ac4_collect,1.278737,1,64b7288026efc4a12f4e3351f03f2ac4,collect,-21.105158,-47.785773
4,64b7288026efc4a12f4e3351f03f2ac4,-22.147359,-48.526709,delivery,64b7288026efc4a12f4e3351f03f2ac4_deliveryMan,2.198454,2,64b7288026efc4a12f4e3351f03f2ac4,deliveryMan,-23.115526,-46.552918
3,64b7288026efc4a12f4e3351f03f2ac4,-23.115526,-46.552918,deliveryMan,64b7288026efc4a12f4e3351f03f2ac4_delivery,2.198454,1,64b7288026efc4a12f4e3351f03f2ac4,delivery,-22.147359,-48.526709
1,64b7288026efc4a12f4e3351f03f2ac4,-23.115526,-46.552918,deliveryMan,64b7288026efc4a12f4e3351f03f2ac4_collect,2.358286,2,64b7288026efc4a12f4e3351f03f2ac4,collect,-21.105158,-47.785773


In [25]:
test_deliveryMan = test_2.query("type == 'deliveryMan'").sort_values(by=["value"], ascending=True).iloc[:1]

test_3 = pd.concat([test_deliveryMan,
                    test_2.query("type != 'deliveryMan'")
                   ])

test_3

,Id,lat_x,long_x,type,variable,value,rw,Id_neighbor,type_neighbor,lat_y,long_y
3,64b7288026efc4a12f4e3351f03f2ac4,-23.115526,-46.552918,deliveryMan,64b7288026efc4a12f4e3351f03f2ac4_delivery,2.198454,1,64b7288026efc4a12f4e3351f03f2ac4,delivery,-22.147359,-48.526709
0,64b7288026efc4a12f4e3351f03f2ac4,-22.147359,-48.526709,delivery,64b7288026efc4a12f4e3351f03f2ac4_collect,1.278737,1,64b7288026efc4a12f4e3351f03f2ac4,collect,-21.105158,-47.785773
2,64b7288026efc4a12f4e3351f03f2ac4,-21.105158,-47.785773,collect,64b7288026efc4a12f4e3351f03f2ac4_delivery,1.278737,1,64b7288026efc4a12f4e3351f03f2ac4,delivery,-22.147359,-48.526709
4,64b7288026efc4a12f4e3351f03f2ac4,-22.147359,-48.526709,delivery,64b7288026efc4a12f4e3351f03f2ac4_deliveryMan,2.198454,2,64b7288026efc4a12f4e3351f03f2ac4,deliveryMan,-23.115526,-46.552918
5,64b7288026efc4a12f4e3351f03f2ac4,-21.105158,-47.785773,collect,64b7288026efc4a12f4e3351f03f2ac4_deliveryMan,2.358286,2,64b7288026efc4a12f4e3351f03f2ac4,deliveryMan,-23.115526,-46.552918


In [26]:
plot_map(test_3, zoom_start=8)

# Teste da aplicação

In [3]:
import sys
sys.path.insert(0, "..\\..\\Delivery-Router\\")
import GraphGenerator as gr
pd.options.mode.chained_assignment = None

In [21]:
orders = pd.read_csv("..\\..\\Delivery-Clustering\\Notebooks\\Bases\\04_Resultado_da_Clusterização_e_Matching.csv")
orders.query("cluster == 1")

,Id,TamanhoPacote__c,CollectPoint__Latitude__s,CollectPoint__Longitude__s,DeliveryPoint__Latitude__s,DeliveryPoint__Longitude__s,cluster,id_deliveryman,car_capacity__c,DeliveryManLocation__Latitude__s,DeliveryManLocation__Longitude__s
4,44f77123c08b319dff9a40cb561a6169,Pequeno,-23.718206,-46.692554,-20.647806,-50.225672,1,96477,Baixa,-23.595924,-46.669864
17,56a726fa0f40d3678696e62aaed9690d,Pequeno,-23.480948,-46.589204,-20.269329,-50.548525,1,96477,Baixa,-23.595924,-46.669864
41,b474873efc88629c172f639f9b47138d,Pequeno,-23.593297,-46.552484,-21.218322,-50.400362,1,96477,Baixa,-23.595924,-46.669864


In [19]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [26]:
graphs = gr.run(df=orders.query("cluster == 0"))
graphs#.query("origin_id == 'f338453ca69a4ab210fb2f6184e7f6ce_deliveryMan'").T

destination_id,origin_id,type,cluster,1258254c11dace819eb300c703d74d50_collect,1258254c11dace819eb300c703d74d50_delivery,1258254c11dace819eb300c703d74d50_deliveryMan,1595ad12e30eb87fdcd5fc501f7f93e9_collect,1595ad12e30eb87fdcd5fc501f7f93e9_delivery,1595ad12e30eb87fdcd5fc501f7f93e9_deliveryMan,2b654564901d04f0e6aa508f39b120f1_collect,2b654564901d04f0e6aa508f39b120f1_delivery,2b654564901d04f0e6aa508f39b120f1_deliveryMan,2da17c10595f4473acf4564e3bd57ce2_collect,2da17c10595f4473acf4564e3bd57ce2_delivery,2da17c10595f4473acf4564e3bd57ce2_deliveryMan,34fa4dce8775b3b4accea9779a4a6562_collect,34fa4dce8775b3b4accea9779a4a6562_delivery,34fa4dce8775b3b4accea9779a4a6562_deliveryMan,62ffb8b5fc46e6cba7ccb763025fd09c_collect,62ffb8b5fc46e6cba7ccb763025fd09c_delivery,62ffb8b5fc46e6cba7ccb763025fd09c_deliveryMan,679d507dacd35ecf587d994177d06871_collect,679d507dacd35ecf587d994177d06871_delivery,679d507dacd35ecf587d994177d06871_deliveryMan,74e801561512a92bdc060773683b7a54_collect,74e801561512a92bdc060773683b7a54_delivery,74e801561512a92bdc060773683b7a54_deliveryMan,790a65965d2b4ee225f616bd1b7dcd64_collect,790a65965d2b4ee225f616bd1b7dcd64_delivery,790a65965d2b4ee225f616bd1b7dcd64_deliveryMan,7a275273bce174382bf22826d9150592_collect,7a275273bce174382bf22826d9150592_delivery,7a275273bce174382bf22826d9150592_deliveryMan,8831eaa951f7563404fa3355d0a89f1e_collect,8831eaa951f7563404fa3355d0a89f1e_delivery,8831eaa951f7563404fa3355d0a89f1e_deliveryMan,89b0e57c47e090334a13fad8d382246e_collect,89b0e57c47e090334a13fad8d382246e_delivery,89b0e57c47e090334a13fad8d382246e_deliveryMan,8ce09fabe89667119f813bcaa531e675_collect,8ce09fabe89667119f813bcaa531e675_delivery,8ce09fabe89667119f813bcaa531e675_deliveryMan,8ebaf041cc8d84dc5436959bab95f2ce_collect,8ebaf041cc8d84dc5436959bab95f2ce_delivery,8ebaf041cc8d84dc5436959bab95f2ce_deliveryMan,90e4d4f7e2221bf7f81a4873e3161b96_collect,90e4d4f7e2221bf7f81a4873e3161b96_delivery,90e4d4f7e2221bf7f81a4873e3161b96_deliveryMan,9340ec2a1f243ae60a2b70696db3b202_collect,9340ec2a1f243ae60a2b70696db3b202_delivery,9340ec2a1f243ae60a2b70696db3b202_deliveryMan,9abfcdf89ba71476090b2c80fb6ecada_collect,9abfcdf89ba71476090b2c80fb6ecada_delivery,9abfcdf89ba71476090b2c80fb6ecada_deliveryMan,a3f77b12916a4990f41ebed970bcdb92_collect,a3f77b12916a4990f41ebed970bcdb92_delivery,a3f77b12916a4990f41ebed970bcdb92_deliveryMan,a5c681209e1bcb90066e530c285ce2c5_collect,a5c681209e1bcb90066e530c285ce2c5_delivery,a5c681209e1bcb90066e530c285ce2c5_deliveryMan,a640d10af396f0aa5f1c751f1f976bdc_collect,a640d10af396f0aa5f1c751f1f976bdc_delivery,a640d10af396f0aa5f1c751f1f976bdc_deliveryMan,a76ada6fa5bcfc4b6225ec88c219ab99_collect,a76ada6fa5bcfc4b6225ec88c219ab99_delivery,a76ada6fa5bcfc4b6225ec88c219ab99_deliveryMan,b1e5cc374b5c4f7c12ef420d322b07e1_collect,b1e5cc374b5c4f7c12ef420d322b07e1_delivery,b1e5cc374b5c4f7c12ef420d322b07e1_deliveryMan,b4b1a36c4c7b2eefca47b21df8d5d535_collect,b4b1a36c4c7b2eefca47b21df8d5d535_delivery,b4b1a36c4c7b2eefca47b21df8d5d535_deliveryMan,b8fc541ddf448e0a01b133c5f34947dd_collect,b8fc541ddf448e0a01b133c5f34947dd_delivery,b8fc541ddf448e0a01b133c5f34947dd_deliveryMan,bb0c84a759545be3fa2155ad0d43444d_collect,bb0c84a759545be3fa2155ad0d43444d_delivery,bb0c84a759545be3fa2155ad0d43444d_deliveryMan,bb66054173f41d6e29497808a82a614a_collect,bb66054173f41d6e29497808a82a614a_delivery,bb66054173f41d6e29497808a82a614a_deliveryMan,e4313fe2f9d83782c9426892dc497ea4_collect,e4313fe2f9d83782c9426892dc497ea4_delivery,e4313fe2f9d83782c9426892dc497ea4_deliveryMan,e6a4c64b81f572a51ecc005e12ccb92d_collect,e6a4c64b81f572a51ecc005e12ccb92d_delivery,e6a4c64b81f572a51ecc005e12ccb92d_deliveryMan,e99d985e66dc0f62859946b86b5beaff_collect,e99d985e66dc0f62859946b86b5beaff_delivery,e99d985e66dc0f62859946b86b5beaff_deliveryMan,f338453ca69a4ab210fb2f6184e7f6ce_collect,f338453ca69a4ab210fb2f6184e7f6ce_delivery,f338453ca69a4ab210fb2f6184e7f6ce_deliveryMan,f9021afbb4a99d23f8e0adc2c702ad95_collect,f9021afbb4a99d23f8e0adc2c702ad95_delivery,f9021afbb4a99d23f8e0adc2c702ad95_delive

In [58]:
from random import randint

randint(2, 7)

5

In [94]:
orders.query(f"Id == '32ef444756734af84aa60ffb0e56e391'")[["DeliveryManLocation__Latitude__s", "DeliveryManLocation__Longitude__s", "id_deliveryman"]].values[0][2]

# pd.Series.values

165215.0